In [ ]:
import os
import hashlib
class LoginAlreadyTakenError(Exception):
    """Login already taken"""
    pass
class PasswordLengthInputError(Exception):
    """Password should be at least 10 characters and at most 50""" 
    pass
class PasswordCharacterInputError(Exception):
    """"Password cannot contain" + ["&", "|", "%", "=","<",">", "/","\"","+"]"""
    pass
class PasswordStrenghtError(Exception):
    """"Password must contain at least one uppercase character, one number and one special character"""
    pass
class InvalidLoginError(Exception):
    """"No such login"""
    pass

Database=[] #this represents sql database
SiteSalts=[] # this represents salts known only to the website
AutenticatedMails=[{"Login" : "sample", "tempPas": "2kmn!Lkm!@AJ230" }] #list of mails viable for an account

In [ ]:

def Register(loginFieldString,passwordFieldString,temporaryPasswordFieldString): #the fields are inputed by the user on the webiste while registering
  #check if the user can create an account
  for user in Database:
     if loginFieldString == user["Login"]:
      print("Account already exists")
      return False
  authenticated=False
  for user in AutenticatedMails:
    if user["Login"]==loginFieldString and user["tempPas"]==temporaryPasswordFieldString:
      authenticated=True
      user=None
      break
  if authenticated==False:
    print("This e-mail was not authenticated to create an account or invalid temporary password")
    return False

#checking input
  

  
  if len(passwordFieldString)<10 or len(passwordFieldString)>50 :
    print("Registration failed")
    raise PasswordLengthInputError
  flag=0
  for special in ["~","!", "?", "@", "#" , "$",  "^", "&" ,"*" ,"_" ,"-","(", ")", "[" , "]" , "{ " , "}" , "'" , "." , ",", ":"]:
    if special in passwordFieldString:
      flag=flag+1
      break
  for uppercase in ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']:
    if uppercase in passwordFieldString:
      flag=flag+1
      break
  for number in ["0","1","2","3","4","5","6","7","8","9"]:
    if number in passwordFieldString:
      flag=flag+1
      break
  if flag !=3:
    print("Registration failed")
    raise PasswordStrenghtError

  for symbol in ["&", "|", "%", "=","<",">", "/","\"","+"]:
    if symbol in passwordFieldString: ## this is just an additional measure, those characters should be escaped in both login and password fields already
      print("Registration failed")
      raise PasswordCharacterInputError

# hashing password
  def Hash(password):
    salt = os.urandom(32)
    hashed_password = hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 692137)
    hex_hash = hashed_password.hex()
    return [hex_hash,salt]

  hash=Hash(passwordFieldString)
  #creating user data to be added to database and salt to be added to website
  UserData={"Login" : loginFieldString, "Password" :hash[0]}
  SaltData={"Login" : loginFieldString,"Salt":hash[1]}
  SiteSalts.append(SaltData)
  print("Registration sucessful")
  return UserData

In [ ]:
#user data added to database
Database.append(Register("sample","Sample123!","2kmn!Lkm!@AJ230"))

Registration sucessful


[{'Login': 'sample', 'Password': '9344df57480f31deb963c604602ef422c99d894c6232da416470817e856cf257'}, {'Login': 'sample', 'Password': '47bb8ed5fc092491d01974bad6b59bb67d016d7ca4cdfea8bfcc2de6906c4acb'}, {'Login': 'sample', 'Password': '12ab8f9354aa61f5828d10d347a34001aa13ad45a2c18f3ff155b41efac99359'}]


In [ ]:
## User log in

def Login(loginFieldString,passwordFieldString):
    selectedPerson=None
    for person in Database:
      if person["Login"]==loginFieldString:
        selectedPerson=person
        break
    if selectedPerson==None:
      raise InvalidLoginError
    for salt in SiteSalts:
      if salt["Login"]==loginFieldString:
        selectedSalt=salt 
        break
    hashed_password = hashlib.pbkdf2_hmac('sha256', passwordFieldString.encode(), selectedSalt["Salt"], 692137)
    hashed_password.hex()
    if hashed_password.hex()==selectedPerson["Password"]: 
      print("Log in sucessful")
      return True
    else:
      print("Invalid password")
      return False

In [ ]:
Login("sample","Sample123!") # previously registered user tries to log in

Log in sucessful


True